# Vaizdų klasifikavimas

In [1]:
# Reikalingi paketai
import numpy as np
import torch
from torchvision import models, transforms
from PIL import Image

In [2]:
# Patikrinama, ar yra GPU
print(torch.cuda.is_available())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

False
cpu


In [3]:
# Modelio atsisiuntimas
model = models.vgg16(weights = 'DEFAULT').to(device)
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\viliu/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:48<00:00, 11.5MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [24]:
# Transformacijos
transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])
# transformations = models.VGG16_Weights.DEFAULT.transforms

In [32]:
# Vaizdo nuskaitymas iš disko
img = Image.open('data/airplane/images/0328403566220ed1.jpg')

In [33]:
print(np.array(img).shape)
print(transformations(img).unsqueeze(0).shape)

(683, 1024, 3)
torch.Size([1, 3, 224, 224])


In [34]:
# Modelio paleidimas
output = model(transformations(img).unsqueeze(0).to(device))
print(output.shape)
print(torch.argmax(output))
print(output[0, 245:251])

torch.Size([1, 1000])
tensor(404)
tensor([ 0.2490, -4.9758, -0.2689, -2.2052, -1.9475, -2.3001],
       grad_fn=<SliceBackward0>)


In [30]:
!wget https://mif.vu.lt/~mif28413/gmm/imagenet_classes.txt

--2025-02-21 16:46:59--  https://mif.vu.lt/~mif28413/gmm/imagenet_classes.txt
Resolving mif.vu.lt (mif.vu.lt)... 193.219.42.2
Connecting to mif.vu.lt (mif.vu.lt)|193.219.42.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://klevas.mif.vu.lt/~mif28413/gmm/imagenet_classes.txt [following]
--2025-02-21 16:46:59--  https://klevas.mif.vu.lt/~mif28413/gmm/imagenet_classes.txt
Resolving klevas.mif.vu.lt (klevas.mif.vu.lt)... 193.219.42.12
Connecting to klevas.mif.vu.lt (klevas.mif.vu.lt)|193.219.42.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: 'imagenet_classes.txt.3'

     0K .......... .......... .                               100% 9,79M=0,002s

2025-02-21 16:46:59 (9,79 MB/s) - 'imagenet_classes.txt.3' saved [21675/21675]



In [35]:
# Žmogui suprantama klasė
with open('imagenet_classes.txt', 'r') as fid:
  class_names = fid.readlines()
print(class_names[torch.argmax(output)])

airliner


In [ ]:
# CPU vs GPU